In [5]:
import os
import json
from dotenv import load_dotenv
from supabase.client import Client, create_client
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

load_dotenv()

True

In [7]:
supabase: Client = create_client(
    os.environ.get("SUPABASE_URL"), 
    os.environ.get("SUPABASE_SERVICE_KEY"))

embeddings = HuggingFaceEmbeddings(model_name="Alibaba-NLP/gte-modernbert-base")

/home/kpatelis/projects/gaia/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
with open('metadata.jsonl', 'r') as jsonl_file:
    json_list = list(jsonl_file)

documents = []
for json_str in json_list:
    json_data = json.loads(json_str)
    content = f"Question : {json_data['Question']}\n\nFinal answer : {json_data['Final answer']}"
    embedding = embeddings.embed_query(content)
    document = {
        "content" : content,
        "metadata" : {
            "source" : json_data['task_id']
        },
        "embedding" : embedding,
    }
    documents.append(document)

/home/kpatelis/projects/gaia/.venv/lib/python3.13/site-packages/torch/_dynamo/guards.py:1114: RuntimeWarning: Guards may run slower on Python 3.13.0. Consider upgrading to Python 3.13.1+.
  warnings.warn(
W1212 01:52:44.565000 23688 torch/_inductor/utils.py:1613] [1/0_1] Not enough SMs to use max_autotune_gemm mode
/home/kpatelis/projects/gaia/.venv/lib/python3.13/site-packages/torch/_dynamo/guards.py:1114: RuntimeWarning: Guards may run slower on Python 3.13.0. Consider upgrading to Python 3.13.1+.
  warnings.warn(


In [12]:
# pgvector needs to be enabled, to turn to vector database
# Table needs to be created beforehand in Supabase, with column types
try:
    response = (
        supabase.table("gaia_documents")
        .insert(documents)
        .execute()
    )
except Exception as exception:
    print("Error inserting data into Supabase:", exception)

In [14]:
from langchain_core.tools.retriever import create_retriever_tool